In [12]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [13]:
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

In [14]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam

In [15]:
BATCH_SIZE = 32 # batch 的大小，如果出現 OOM error，請降低這個值
NUM_CLASSES = 5 # Final exam has 5 classes
NUM_EPOCHS = 10 # 訓練的 epochs 數量

DATASET_PATH = "4th-cupoy-ml-100-marathon-finalexam\image_data"
IMAGE_SIZE = (224, 224)

train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/train',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

valid_datagen = ImageDataGenerator()
valid_batches = valid_datagen.flow_from_directory(DATASET_PATH + '/valid',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

Found 2698 images belonging to 5 classes.
Class #0 = daisy
Class #1 = dandelion
Class #2 = rose
Class #3 = sunflower
Class #4 = tulip
Found 125 images belonging to 5 classes.


In [16]:
# from resnet_builder import resnet

# model = resnet(input_shape=(32,32,3), num_classes=NUM_CLASSES) 
# model.summary()

In [17]:
from keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
import numpy as np

FREEZE_LAYERS = 2

# 以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)

# 設定凍結與要進行訓練的網路層
model = Model(inputs=net.input, outputs=output_layer)
for layer in model.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in model.layers[FREEZE_LAYERS:]:
    layer.trainable = True

# 學習率動態調整。當跑到第幾個 epcoh 時，根據設定修改學習率。這邊的數值都是參考原 paper
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-4
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
model.compile(optimizer=Adam(lr=lr_schedule(0)),
              loss='categorical_crossentropy', metrics=['accuracy'])

# 輸出整個網路結構
print(model.summary())

# 使用動態調整學習率
lr_scheduler = LearningRateScheduler(lr_schedule)

# 使用自動降低學習率 (當 validation loss 連續 5 次沒有下降時，自動降低學習率)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
# 設定 callbacks
callbacks = [lr_reducer, lr_scheduler]


Learning rate:  0.0001
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_____________________________________________________________________

In [ ]:
model.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS,
                        callbacks=callbacks)

# history = model.fit(x_train, y_train,
#                     batch_size=batch_size,
#                     epochs=epochs,
#                     verbose=1,
#                     validation_data=(x_test, y_test))

Learning rate:  0.0001
Epoch 1/10
84/84 [==============================] - 1501s 18s/step - loss: 1.5141 - accuracy: 0.6489 - val_loss: 1.6476 - val_accuracy: 0.7708 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 2/10
84/84 [==============================] - 1693s 20s/step - loss: 0.6501 - accuracy: 0.8357 - val_loss: 0.4525 - val_accuracy: 0.8542 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 3/10
84/84 [==============================] - 1596s 19s/step - loss: 0.4789 - accuracy: 0.8657 - val_loss: 0.9835 - val_accuracy: 0.8542 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 4/10
84/84 [==============================] - 1612s 19s/step - loss: 0.3768 - accuracy: 0.8901 - val_loss: 0.4248 - val_accuracy: 0.9062 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 5/10
84/84 [==============================] - 1599s 19s/step - loss: 0.3071 - accuracy: 0.9047 - val_loss: 0.4375 - val_accuracy: 0.9062 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 6/10
43/84 [==============>...............] - ETA: 12:52

In [9]:
from keras.models import load_model
from keras.preprocessing import image

import sys

# 從參數讀取圖檔路徑
# files = sys.argv[1:]

from os import listdir
from os.path import isfile, join
import pandas as pd

# root_path = os.path.dirname(os.path.realpath(__file__))
files_path = "4th-cupoy-ml-100-marathon-finalexam/image_data/test"

files = [[join(files_path, f), f] for f in listdir(files_path) if isfile(join(files_path, f))]

# 載入訓練好的模型
# net = load_model('model-resnet50-final.h5')

cls_list = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

pred_ids = []
pred_classes = []

# 辨識每一張圖
for [f, fname] in files:
    print (f)
    img = image.load_img(f, target_size=(224, 224))
    if img is None:
        continue
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    pred = model.predict(x)[0]
    top_inds = pred.argsort()[::-1][:5]
    print(f)
    for i in top_inds:
        print('    {:.3f}  {}'.format(pred[i], cls_list[i]))
    id = os.path.splitext(fname)[0]
    print(id)
    print(top_inds[0])
    pred_ids.append(id)
    pred_classes.append(top_inds[0])

pred_results = pd.DataFrame({ "id": pred_ids, "flower_class": pred_classes})


4th-cupoy-ml-100-marathon-finalexam/image_data/test\0028624c49b3e0610ff9f1d111f5d532.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\0028624c49b3e0610ff9f1d111f5d532.jpg
    0.448  rose
    0.418  tulip
    0.068  daisy
    0.043  sunflower
    0.023  dandelion
0028624c49b3e0610ff9f1d111f5d532
2
4th-cupoy-ml-100-marathon-finalexam/image_data/test\002c30700185b7971369258b438070d5.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\002c30700185b7971369258b438070d5.jpg
    0.748  tulip
    0.148  rose
    0.051  daisy
    0.049  dandelion
    0.003  sunflower
002c30700185b7971369258b438070d5
4
4th-cupoy-ml-100-marathon-finalexam/image_data/test\00852f4f666acecd0c0d140365b42efd.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\00852f4f666acecd0c0d140365b42efd.jpg
    0.939  tulip
    0.040  rose
    0.020  sunflower
    0.001  daisy
    0.000  dandelion
00852f4f666acecd0c0d140365b42efd
4
4th-cupoy-ml-100-marathon-finalexam/image_data/test\00c08828fce04e360c732cac01edad9e.

4th-cupoy-ml-100-marathon-finalexam/image_data/test\048c3b5db49f963ba9b9ea42ba6b5085.jpg
    0.956  rose
    0.044  tulip
    0.000  daisy
    0.000  sunflower
    0.000  dandelion
048c3b5db49f963ba9b9ea42ba6b5085
2
4th-cupoy-ml-100-marathon-finalexam/image_data/test\0495e5cb93952bc4a728c19e40322679.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\0495e5cb93952bc4a728c19e40322679.jpg
    0.847  tulip
    0.126  rose
    0.026  dandelion
    0.001  daisy
    0.000  sunflower
0495e5cb93952bc4a728c19e40322679
4
4th-cupoy-ml-100-marathon-finalexam/image_data/test\0499d6a2951fe18d4d9ed7b5d908ca19.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\0499d6a2951fe18d4d9ed7b5d908ca19.jpg
    0.929  tulip
    0.071  rose
    0.000  daisy
    0.000  sunflower
    0.000  dandelion
0499d6a2951fe18d4d9ed7b5d908ca19
4
4th-cupoy-ml-100-marathon-finalexam/image_data/test\04b6564507d52dbb8ce0017ea43c9d69.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\04b6564507d52dbb8ce0017ea43c9d69.

4th-cupoy-ml-100-marathon-finalexam/image_data/test\072f62cee23d3ccafb5649cd643bfa36.jpg
    0.998  sunflower
    0.001  tulip
    0.001  dandelion
    0.000  daisy
    0.000  rose
072f62cee23d3ccafb5649cd643bfa36
3
4th-cupoy-ml-100-marathon-finalexam/image_data/test\07473a092e8db87aaa75b0392696b351.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\07473a092e8db87aaa75b0392696b351.jpg
    0.992  sunflower
    0.005  tulip
    0.003  dandelion
    0.000  daisy
    0.000  rose
07473a092e8db87aaa75b0392696b351
3
4th-cupoy-ml-100-marathon-finalexam/image_data/test\088ee0cd81fa231cc740b26711b8f258.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\088ee0cd81fa231cc740b26711b8f258.jpg
    0.579  tulip
    0.420  rose
    0.000  sunflower
    0.000  daisy
    0.000  dandelion
088ee0cd81fa231cc740b26711b8f258
4
4th-cupoy-ml-100-marathon-finalexam/image_data/test\08932a183188af4544502e9ab6c67251.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\08932a183188af4544502e9ab6c67251.

4th-cupoy-ml-100-marathon-finalexam/image_data/test\0bed759db7aa5774a97406e2c2621769.jpg
    0.994  sunflower
    0.005  tulip
    0.001  dandelion
    0.000  daisy
    0.000  rose
0bed759db7aa5774a97406e2c2621769
3
4th-cupoy-ml-100-marathon-finalexam/image_data/test\0bfd70207c02f6fc77f4835468024c2a.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\0bfd70207c02f6fc77f4835468024c2a.jpg
    0.791  tulip
    0.209  rose
    0.000  sunflower
    0.000  daisy
    0.000  dandelion
0bfd70207c02f6fc77f4835468024c2a
4
4th-cupoy-ml-100-marathon-finalexam/image_data/test\0c09c85e41c0b33d227f682fcf6d668c.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\0c09c85e41c0b33d227f682fcf6d668c.jpg
    0.895  tulip
    0.093  rose
    0.012  sunflower
    0.000  daisy
    0.000  dandelion
0c09c85e41c0b33d227f682fcf6d668c
4
4th-cupoy-ml-100-marathon-finalexam/image_data/test\0c0d490e02aac9ef73c63ba9732c05b6.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\0c0d490e02aac9ef73c63ba9732c05b6.

4th-cupoy-ml-100-marathon-finalexam/image_data/test\0e69adb07b3305ffe605632d46b706c9.jpg
    0.584  sunflower
    0.155  dandelion
    0.154  daisy
    0.066  tulip
    0.040  rose
0e69adb07b3305ffe605632d46b706c9
3
4th-cupoy-ml-100-marathon-finalexam/image_data/test\0e6eff26cf27ea98c32cfe666c7fc600.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\0e6eff26cf27ea98c32cfe666c7fc600.jpg
    0.527  tulip
    0.384  rose
    0.041  daisy
    0.039  sunflower
    0.009  dandelion
0e6eff26cf27ea98c32cfe666c7fc600
4
4th-cupoy-ml-100-marathon-finalexam/image_data/test\0e75e51a1b2c85ae8ea00e5c8d260465.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\0e75e51a1b2c85ae8ea00e5c8d260465.jpg
    0.918  tulip
    0.062  sunflower
    0.018  rose
    0.001  daisy
    0.000  dandelion
0e75e51a1b2c85ae8ea00e5c8d260465
4
4th-cupoy-ml-100-marathon-finalexam/image_data/test\0e90cb853133df4634bb1ac2cd8d7b4e.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\0e90cb853133df4634bb1ac2cd8d7b4e.

...

4th-cupoy-ml-100-marathon-finalexam/image_data/test\f4252aa28723e9e240c37cb4783886c4.jpg
    0.803  tulip
    0.103  rose
    0.085  sunflower
    0.006  daisy
    0.003  dandelion
f4252aa28723e9e240c37cb4783886c4
4
4th-cupoy-ml-100-marathon-finalexam/image_data/test\f4863bc3a15b27de20a60ac764c6fafa.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\f4863bc3a15b27de20a60ac764c6fafa.jpg
    0.673  sunflower
    0.142  tulip
    0.136  dandelion
    0.035  daisy
    0.015  rose
f4863bc3a15b27de20a60ac764c6fafa
3
4th-cupoy-ml-100-marathon-finalexam/image_data/test\f48df25f38581b5ac319f5722875c897.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\f48df25f38581b5ac319f5722875c897.jpg
    0.643  tulip
    0.243  sunflower
    0.065  dandelion
    0.046  rose
    0.005  daisy
f48df25f38581b5ac319f5722875c897
4
4th-cupoy-ml-100-marathon-finalexam/image_data/test\f49e74d9cc932fa8f2caa90f17a4f0c6.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\f49e74d9cc932fa8f2caa90f17a4f0c6.

4th-cupoy-ml-100-marathon-finalexam/image_data/test\f6978ff0f46a1db7c7a380750b491048.jpg
    0.918  tulip
    0.075  rose
    0.007  sunflower
    0.000  daisy
    0.000  dandelion
f6978ff0f46a1db7c7a380750b491048
4
4th-cupoy-ml-100-marathon-finalexam/image_data/test\f69a90c5abddebc1382f0868519028b9.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\f69a90c5abddebc1382f0868519028b9.jpg
    0.949  dandelion
    0.029  rose
    0.022  tulip
    0.000  daisy
    0.000  sunflower
f69a90c5abddebc1382f0868519028b9
1
4th-cupoy-ml-100-marathon-finalexam/image_data/test\f6a224537c270d497307caacfe163881.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\f6a224537c270d497307caacfe163881.jpg
    0.703  sunflower
    0.166  dandelion
    0.066  rose
    0.033  daisy
    0.032  tulip
f6a224537c270d497307caacfe163881
3
4th-cupoy-ml-100-marathon-finalexam/image_data/test\f707cdb7294afcf25a74ac58f681c15e.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\f707cdb7294afcf25a74ac58f681c15e.

4th-cupoy-ml-100-marathon-finalexam/image_data/test\f9745be00ab758aff6cb7272d1d6d625.jpg
    0.470  dandelion
    0.189  rose
    0.167  daisy
    0.096  tulip
    0.078  sunflower
f9745be00ab758aff6cb7272d1d6d625
1
4th-cupoy-ml-100-marathon-finalexam/image_data/test\f97500a77bb3297b433b6c86f603a3b4.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\f97500a77bb3297b433b6c86f603a3b4.jpg
    0.889  tulip
    0.111  rose
    0.000  daisy
    0.000  sunflower
    0.000  dandelion
f97500a77bb3297b433b6c86f603a3b4
4
4th-cupoy-ml-100-marathon-finalexam/image_data/test\f9aadb1a44b9666f5d996f5b4049e45f.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\f9aadb1a44b9666f5d996f5b4049e45f.jpg
    0.679  tulip
    0.308  rose
    0.007  sunflower
    0.005  daisy
    0.000  dandelion
f9aadb1a44b9666f5d996f5b4049e45f
4
4th-cupoy-ml-100-marathon-finalexam/image_data/test\f9b5254ddc08b637e3eaefb58dd5ba79.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\f9b5254ddc08b637e3eaefb58dd5ba79.

4th-cupoy-ml-100-marathon-finalexam/image_data/test\fd7b7e558cd63f0cf26025b63d3b63a1.jpg
    0.889  tulip
    0.077  sunflower
    0.026  rose
    0.006  daisy
    0.002  dandelion
fd7b7e558cd63f0cf26025b63d3b63a1
4
4th-cupoy-ml-100-marathon-finalexam/image_data/test\fd9cac41cda00325613b362025eb9cc1.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\fd9cac41cda00325613b362025eb9cc1.jpg
    0.596  sunflower
    0.189  tulip
    0.086  dandelion
    0.079  daisy
    0.050  rose
fd9cac41cda00325613b362025eb9cc1
3
4th-cupoy-ml-100-marathon-finalexam/image_data/test\fda6dca87a0de48f83227817e3421e20.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\fda6dca87a0de48f83227817e3421e20.jpg
    0.943  tulip
    0.044  rose
    0.008  dandelion
    0.004  sunflower
    0.001  daisy
fda6dca87a0de48f83227817e3421e20
4
4th-cupoy-ml-100-marathon-finalexam/image_data/test\fda7085f5d8b6f995922b19f5aeec873.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\fda7085f5d8b6f995922b19f5aeec873.

4th-cupoy-ml-100-marathon-finalexam/image_data/test\ffea1f275c05accb0a6bfd1203620c7e.jpg
    0.659  sunflower
    0.180  tulip
    0.077  dandelion
    0.048  daisy
    0.036  rose
ffea1f275c05accb0a6bfd1203620c7e
3
4th-cupoy-ml-100-marathon-finalexam/image_data/test\ffeb2a1cf53464b6af937ab8af0c2946.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\ffeb2a1cf53464b6af937ab8af0c2946.jpg
    0.921  sunflower
    0.058  tulip
    0.017  dandelion
    0.003  daisy
    0.001  rose
ffeb2a1cf53464b6af937ab8af0c2946
3
4th-cupoy-ml-100-marathon-finalexam/image_data/test\ffefcc68e2e7eed8b17b0b5b0f740538.jpg
4th-cupoy-ml-100-marathon-finalexam/image_data/test\ffefcc68e2e7eed8b17b0b5b0f740538.jpg
    0.883  tulip
    0.053  rose
    0.041  dandelion
    0.013  sunflower
    0.009  daisy
ffefcc68e2e7eed8b17b0b5b0f740538
4


In [11]:
pred_results.to_csv('sample_submission.csv', index=False)